<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Seed_VC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -q https://huggingface.co/spaces/Plachta/Seed-VC app
%cd app
!sed -i "s/).launch()/).launch(share=True)/" app.py
!pip install --quiet -r  requirements.txt && pip install --quiet protobuf tensorflow numpy gradio spaces
!python app.py